In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statistics import mean,stdev

In [15]:
def standard_normal_distribution(x,avg,std):
    return np.exp(-0.5*(x-avg)*(x-avg)/(std*std))/(np.sqrt(2*np.pi))

In [72]:
def evaluation_summary(label,model_metrics):
    print(f"Model Name: {label}\n")
    print(f"Accuracy: {model_metrics['Accuracy']}\n")

    if len(model_metrics):
        print(f"Precision: {model_metrics['Precision']}\n")
        print(f"Recall: {model_metrics['Recall']}\n")
        print(f"F1_Score: {model_metrics['F1_score']}\n")
    print(f"Confusion_Matrix:\n {model_metrics['Confusion_Matrix']}")

In [73]:
def Eval_metrics(y_test,y_pred,num_classes):

    confusion_matrix=np.zeros((num_classes,num_classes))
    for i in range(y_test.shape[0]):
        confusion_matrix[y_pred[i]][y_test[i]]=confusion_matrix[y_pred[i]][y_test[i]]+1




    accuracy=0;
    for i in range(num_classes):
        accuracy=accuracy+confusion_matrix[i][i]
    accuracy=accuracy/y_test.shape[0]



    return {'Accuracy':accuracy,'Confusion_Matrix':confusion_matrix}

In [74]:
def Eval_metrics_naive_bayes(y_test,y_pred):
    tp=0
    fp=0
    tn=0
    fn=0
    for i in range(y_test.shape[0]):
        if y_test[i]==y_pred[i] and y_test[i]==1:
            tp=tp+1
        elif y_test[i]==y_pred[i] and y_test[i]==0:
            tn=tn+1
        elif y_test[i]!=y_pred[i] and y_pred[i]==1:
            fp=fp+1
        else:
            fn=fn+1

    accuracy=(tn+tp)/(tn+tp+fn+fp)
    precision= tp / (tp + fp)
    recall=tp/(tp+fn)
    f1_score=2*precision*recall/(precision+recall)
    confusion_matrix=[(tp,fp),(fn,tn)]
    df=pd.DataFrame(confusion_matrix)
    df.index.name=None

    return {'Accuracy':accuracy,'Precision':precision,'Recall':recall,'F1_score':f1_score,'Confusion_Matrix':df}

**Part A** - *Naive Bayes Classifier to Predict Income*

Task 1- *Data Preprocessing*

In [18]:
df=pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",index_col=False,names=["age","workclass","fnlwgt","education","education-num","marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country",">=50K"])
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,>=50K
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  >=50K           32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [20]:
print(df['race'].unique(), '\n', df['race'].nunique())

[' White' ' Black' ' Asian-Pac-Islander' ' Amer-Indian-Eskimo' ' Other'] 
 5


In [21]:
df['race']=df['race'].replace([' Black', " White", ' Asian-Pac-Islander', ' Other', ' Amer-Indian-Eskimo'],
                              [1, 2, 3, 4, 5]
                              )

In [22]:
print(df['marital-status'].unique(), '\n', df['marital-status'].nunique())

[' Never-married' ' Married-civ-spouse' ' Divorced'
 ' Married-spouse-absent' ' Separated' ' Married-AF-spouse' ' Widowed'] 
 7


In [23]:
df['marital-status']=df['marital-status'].replace([' Never-married', ' Married-civ-spouse', ' Divorced'
                                                   ,' Married-spouse-absent', ' Separated', ' Married-AF-spouse', ' Widowed'],
                              [1, 2, 3, 4, 5,6,7]
                              )

In [24]:
print(df['workclass'].unique(), '\n', df['workclass'].nunique())

[' State-gov' ' Self-emp-not-inc' ' Private' ' Federal-gov' ' Local-gov'
 ' ?' ' Self-emp-inc' ' Without-pay' ' Never-worked'] 
 9


In [25]:
df['workclass'].replace([' Private', ' Local-gov', ' ?', ' Self-emp-not-inc', ' Federal-gov', ' State-gov',
                         ' Self-emp-inc', ' Without-pay', ' Never-worked'], [1, 2, 3, 4, 5, 6, 7, 8, 9], inplace=True)

In [26]:
print(df['education'].unique(), '\n',df['occupation'].nunique())

[' Bachelors' ' HS-grad' ' 11th' ' Masters' ' 9th' ' Some-college'
 ' Assoc-acdm' ' Assoc-voc' ' 7th-8th' ' Doctorate' ' Prof-school'
 ' 5th-6th' ' 10th' ' 1st-4th' ' Preschool' ' 12th'] 
 15


In [27]:
df['education'].replace([' 11th', ' HS-grad', ' Assoc-acdm', ' Some-college', ' 10th', ' Prof-school',
                         ' 7th-8th', ' Bachelors', ' Masters', ' Doctorate', ' 5th-6th', ' Assoc-voc', ' 9th',
                         ' 12th', ' 1st-4th', ' Preschool'], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], inplace=True)

In [28]:
print(df['occupation'].unique(), "\n" ,df['occupation'].nunique())

[' Adm-clerical' ' Exec-managerial' ' Handlers-cleaners' ' Prof-specialty'
 ' Other-service' ' Sales' ' Craft-repair' ' Transport-moving'
 ' Farming-fishing' ' Machine-op-inspct' ' Tech-support' ' ?'
 ' Protective-serv' ' Armed-Forces' ' Priv-house-serv'] 
 15


In [29]:
df['occupation'].replace([' Machine-op-inspct', ' Farming-fishing', ' Protective-serv', ' ?',
                          ' Other-service', ' Prof-specialty', ' Craft-repair', ' Adm-clerical',
                          ' Exec-managerial', ' Tech-support', ' Sales', ' Priv-house-serv',
                          ' Transport-moving', ' Handlers-cleaners', ' Armed-Forces'],
                         [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], inplace=True
                         )

In [30]:
print(df['relationship'].unique(), '\n', df['relationship'].nunique())

[' Not-in-family' ' Husband' ' Wife' ' Own-child' ' Unmarried'
 ' Other-relative'] 
 6


In [31]:
df['relationship'].replace([' Own-child', ' Husband', ' Not-in-family', ' Unmarried', ' Wife', ' Other-relative'],
                           [1, 2, 3, 4, 5, 6], inplace=True)

In [32]:
print(df['sex'].unique(), '\n', df['sex'].nunique())

[' Male' ' Female'] 
 2


In [33]:
df['sex'].replace([' Male', ' Female'], [0, 1], inplace=True)

In [34]:
l = []
for i in range(42):
    l.append(i)

In [35]:
df['native-country'].replace([' United-States', ' ?', ' Peru', ' Guatemala', ' Mexico', ' Dominican-Republic',
                              ' Ireland', ' Germany', ' Philippines', ' Thailand', ' Haiti', ' El-Salvador',
                              ' Puerto-Rico', ' Vietnam', ' South', ' Columbia', ' Japan', ' India', ' Cambodia',
                              ' Poland', ' Laos', ' England', ' Cuba', ' Taiwan', ' Italy', ' Canada', ' Portugal',
                              ' China', ' Nicaragua', ' Honduras', ' Iran', ' Scotland', ' Jamaica', ' Ecuador',
                              ' Yugoslavia', ' Hungary', ' Hong', ' Greece', ' Trinadad&Tobago',
                              ' Outlying-US(Guam-USVI-etc)', ' France', ' Holand-Netherlands'], l, inplace=True)

In [36]:
print(df['native-country'].unique(), '\n', df['native-country'].nunique())

[ 0 22 32 17  1  4 14 12 29 21 25  7 30  8 24 19 15 18  9 33 20 23 10 26
  5 11 40  3 27 16 34  2 39 31 38 37 28 13 36  6 35 41] 
 42


In [37]:
df['>=50K'].replace([' <=50K', ' >50K'], [0, 1], inplace=True)

In [38]:
df["age"]=(df["age"]-df["age"].mean())/df["age"].std()
df["fnlwgt"]=(df["fnlwgt"]-df["fnlwgt"].mean())/df["fnlwgt"].std()
df["education-num"]=(df["education-num"]-df["education-num"].mean())/df["education-num"].std()
df["capital-gain"]=(df["capital-gain"]-df["capital-gain"].mean())/df["capital-gain"].std()
df["capital-loss"]=(df["capital-loss"]-df["capital-loss"].mean())/df["capital-loss"].std()
df["hours-per-week"]=(df["hours-per-week"]-df["hours-per-week"].mean())/df["hours-per-week"].std()

In [39]:
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,>=50K
0,0.030670,6,-1.063594,8,1.134721,1,8,3,2,0,0.148451,-0.216656,-0.035429,0,0
1,0.837096,4,-1.008692,8,1.134721,2,9,2,2,0,-0.145918,-0.216656,-2.222119,0,0
2,-0.042641,1,0.245075,2,-0.420053,3,14,3,2,0,-0.145918,-0.216656,-0.035429,0,0
3,1.057031,1,0.425795,1,-1.197440,2,14,2,1,0,-0.145918,-0.216656,-0.035429,0,0
4,-0.775756,1,1.408154,8,1.134721,2,6,5,1,1,-0.145918,-0.216656,-0.035429,22,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,-0.849067,1,0.639731,3,0.746028,2,10,5,2,1,-0.145918,-0.216656,-0.197406,0,0
32557,0.103982,1,-0.335428,2,-0.420053,2,1,2,2,0,-0.145918,-0.216656,-0.035429,0,1
32558,1.423588,1,-0.358772,2,-0.420053,7,8,4,2,1,-0.145918,-0.216656,-0.035429,0,0
32559,-1.215625,1,0.110958,2,-0.420053,1,8,1,2,0,-0.145918,-0.216656,-1.655199,0,0


Task 2

In [40]:
train_data=df.sample(frac=0.67,random_state=42)
test_data=pd.concat([df,train_data]).drop_duplicates(keep=False)
X_train=train_data.drop('>=50K',axis=1).values
Y_train=train_data['>=50K'].values
X_test=test_data.drop('>=50K',axis=1).values
Y_test=test_data['>=50K'].values

In [41]:
X_train

array([[-0.84906741,  1.        , -0.28043934, ..., -0.2166562 ,
        -0.19740595,  0.        ],
       [ 0.47053884,  6.        , -1.31891422, ..., -0.2166562 ,
        -0.0354289 ,  0.        ],
       [-0.70244449,  1.        , -0.03666857, ..., -0.2166562 ,
         1.17939893,  0.        ],
       ...,
       [ 1.35027633,  1.        ,  1.62525504, ..., -0.2166562 ,
        -0.0354289 ,  0.        ],
       [-0.92237887,  1.        , -1.42836948, ..., -0.2166562 ,
        -0.0354289 ,  5.        ],
       [ 0.25060446,  1.        , -0.16703335, ..., -0.2166562 ,
        -0.0354289 ,  0.        ]])

In [42]:
Continuous_features=[0,2,4,10,11,12]

In [43]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,>=50K
0,0.030670,6,-1.063594,8,1.134721,1,8,3,2,0,0.148451,-0.216656,-0.035429,0,0
1,0.837096,4,-1.008692,8,1.134721,2,9,2,2,0,-0.145918,-0.216656,-2.222119,0,0
2,-0.042641,1,0.245075,2,-0.420053,3,14,3,2,0,-0.145918,-0.216656,-0.035429,0,0
3,1.057031,1,0.425795,1,-1.197440,2,14,2,1,0,-0.145918,-0.216656,-0.035429,0,0
4,-0.775756,1,1.408154,8,1.134721,2,6,5,1,1,-0.145918,-0.216656,-0.035429,22,0


In [44]:
pos_neg_params=[]
for i in Continuous_features:
    pos=[]
    neg=[]
    for j in range(X_train.shape[0]):
        if Y_train[j]==1:
            pos.append(X_train[j][i])
        else:
            neg.append(X_train[j][i])
    temp=[]
    temp.append(mean(pos))
    temp.append(stdev(pos))
    temp.append(mean(neg))
    temp.append(stdev(neg))
    pos_neg_params.append(temp)

In [45]:
def prior_probablity_each_class(Y_train,target):
    count=0
    for i in range(Y_train.shape[0]):
        if Y_train[i]==target:
            count=count+1
    return count/Y_train.shape[0]

In [46]:
def conditional_probability_feature(feature_index,X_train,Y_train,target_class,feature):
    if feature_index in Continuous_features:
        pos=0
        for i in range(len(Continuous_features)):
            if Continuous_features[i]==feature_index:
                pos=i
                break
        num=0
        if target_class==1:
            num=standard_normal_distribution(feature,pos_neg_params[pos][0],pos_neg_params[pos][1])
        else:
            num=standard_normal_distribution(feature,pos_neg_params[pos][2],pos_neg_params[pos][3])
        return num
    else:
        den=0
        count=0
        for i in range(X_train.shape[0]):
            if Y_train[i]==target_class:
                den=den+1
                if X_train[i][feature_index]==feature:
                    count=count+1
        return count/den


In [50]:
def predict(X_train,Y_train,X_test,i):
    p_pos=prior_probablity_each_class(Y_train,1)
    p_neg=prior_probablity_each_class(Y_train,0)
    for j in range(14):
        p_pos=p_pos*conditional_probability_feature(j,X_train,Y_train,1,X_test[i][j])
    for j in range(14):
        p_neg=p_neg*conditional_probability_feature(j,X_train,Y_train,0,X_test[i][j])
    if p_pos>=p_neg:
        return 1
    else:
        return 0

In [69]:
Y_pred=[]
for i in range(len(X_test)):
    Y_pred.append(predict(X_train,Y_train,X_test,i))


In [76]:
metrics=Eval_metrics_naive_bayes(Y_test[:len(X_test)],Y_pred)
evaluation_summary('Naive',metrics)

Model Name: Naive

Accuracy: 0.8093333333333333

Precision: 0.6042553191489362

Recall: 0.7395833333333334

F1_Score: 0.6651053864168619

Confusion_Matrix:
      0    1
0  284  186
1  100  930


# Smoothing techinque

In [66]:
def conditional_probability_feature_with_smoothing(feature_index,X_train,Y_train,target_class,feature):
    if feature_index in Continuous_features:
        pos=0
        for i in range(len(Continuous_features)):
            if Continuous_features[i]==feature_index:
                pos=i
                break

        num=0
        if target_class==1:
            num=standard_normal_distribution(feature,pos_neg_params[pos][0],pos_neg_params[pos][1])
        else:
            num=standard_normal_distribution(feature,pos_neg_params[pos][2],pos_neg_params[pos][3])

        return num
    else:
        den=0
        count=0
        nc=0
        n=0
        m=0
        for i in range(X_train.shape[0]):
            if Y_train[i]==target_class:
                den=den+1
                n=n+1
                if X_train[i][feature_index]==feature:
                    nc=nc+1
                    count=count+1
        if count!=0:
            return count/den
        # To handle 0 probabilities
        m=len(np.unique(df.iloc[:,feature_index]))
        return (nc+1)/(n+m)





In [67]:
Y_pred=[]
for i in range(len(X_test)):
    Y_pred.append(predict(X_train,Y_train,X_test,i))

In [78]:
metrics=Eval_metrics_naive_bayes(Y_test[:len(X_test)],Y_pred)
evaluation_summary('Naive',metrics)

Model Name: Naive

Accuracy: 0.824

Precision: 0.6185897435897436

Recall: 0.772

F1_Score: 0.6868327402135231

Confusion_Matrix:
      0    1
0  193  119
1   57  631


# Logistic Regression

In [86]:
from sklearn.linear_model import LogisticRegression
classifier=LogisticRegression()
classifier.fit(X_train,Y_train)
Y_pred=classifier.predict(X_test)
metrics=Eval_metrics_naive_bayes(Y_pred,Y_test)
evaluation_summary('Logistic Regression',metrics)

Model Name: Logistic Regression

Accuracy: 0.8250372856077554

Precision: 0.4525631216526396

Recall: 0.7262124002455494

F1_Score: 0.5576243224133868

Confusion_Matrix:
       0     1
0  1183  1431
1   446  7668


In [87]:
from sklearn.neighbors import KNeighborsClassifier
classifier=KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train,Y_train)
Y_pred=classifier.predict(X_test)
metrics=Eval_metrics_naive_bayes(Y_pred,Y_test)
evaluation_summary('K-Nearest Neighbours',metrics)

Model Name: K-Nearest Neighbours

Accuracy: 0.8187919463087249

Precision: 0.5734506503442999

Recall: 0.6439003436426117

F1_Score: 0.6066369890732497

Confusion_Matrix:
       0     1
0  1499  1115
1   829  7285


# Part B

Importing Dataset

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.activations import sigmoid,relu,linear,softmax,tanh
from tensorflow.keras.losses import CategoricalCrossentropy,SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import SGD,Adam

In [ ]:
(X_train, Y_train), (X_test, Y_test) = tf.keras.datasets.mnist.load_data()
X_test=tf.constant(X_test)

In [ ]:
X_test.shape

TensorShape([10000, 28, 28])

In [ ]:
X_train.shape,Y_train.shape

((60000, 28, 28), (60000,))

In [ ]:
X_train=tf.constant(X_train)
Y_train=tf.constant(Y_train)

Model 1

In [1]:
model_1=Sequential([
    Flatten(input_shape=(28,28)),
    Dense(150,activation=relu),
    Dense(100,activation=relu),
    Dense(10,activation=softmax)
])

model_1.compile(optimizer='adam',loss=SparseCategoricalCrossentropy(),metrics=["accuracy"])
model_1.fit(X_train,Y_train,epochs=3,verbose=0)

Y_pred=[]
for i in range(X_test.shape[0]):
  num=model_1.predict(tf.expand_dims(X_test[i],0),verbose=0).argmax()
  Y_pred.append(num)

metrics=Eval_metrics(Y_test,Y_pred,10)
evaluation_summary('Model_1',metrics)


In [ ]:
model_2=Sequential([
    Flatten(input_shape=(28,28)),
    Dense(100,activation=relu),
    Dense(150,activation=relu),
    Dense(10,activation=softmax)
])

model_2.compile(optimizer='adam',loss=SparseCategoricalCrossentropy(),metrics=["accuracy"])
model_2.fit(X_train,Y_train,epochs=3,verbose=0)

Y_pred=[]
for i in range(X_test.shape[0]):
    num=model_2.predict(tf.expand_dims(X_test[i],0),verbose=0).argmax()
    Y_pred.append(num)

metrics=Eval_metrics(Y_test,Y_pred,10)
evaluation_summary('Model_2',metrics)


In [ ]:
model_3=Sequential([
    Flatten(input_shape=(28,28)),
    Dense(100,activation=tanh),
    Dense(100,activation=tanh),
    Dense(10,activation=softmax)
])

model_3.compile(optimizer='adam',loss=SparseCategoricalCrossentropy(),metrics=["accuracy"])
model_3.fit(X_train,Y_train,epochs=3,verbose=0)

Y_pred=[]
for i in range(X_test.shape[0]):
    num=model_3.predict(tf.expand_dims(X_test[i],0),verbose=0).argmax()
    Y_pred.append(num)

metrics=Eval_metrics(Y_test,Y_pred,10)
evaluation_summary('Model_3',metrics)


Model Name: Model_3

Accuracy: 0.888

Confusion_Matrix:
 [[ 83.   0.   3.   0.   0.   4.   3.   0.   1.   0.]
 [  0. 125.   0.   1.   0.   0.   0.   1.   0.   1.]
 [  0.   0. 105.   3.   0.   1.   0.   3.   4.   0.]
 [  0.   0.   3.  91.   0.   5.   0.   3.   8.   1.]
 [  0.   0.   0.   0.  88.   0.   0.   1.   1.   3.]
 [  0.   0.   0.   3.   0.  74.   1.   0.   1.   0.]
 [  2.   0.   1.   1.   3.   1.  83.   0.   3.   0.]
 [  0.   0.   1.   2.   0.   0.   0.  85.   1.   1.]
 [  0.   1.   3.   5.   1.   1.   0.   1.  69.   3.]
 [  0.   0.   0.   1.  18.   1.   0.   5.   1.  85.]]


In [ ]:
model_4=Sequential([
    Flatten(input_shape=(28,28)),
    Dense(150,activation=tanh),
    Dense(100,activation=relu),
    Dense(10,activation=softmax)
])

model_4.compile(optimizer='adam',loss=SparseCategoricalCrossentropy(),metrics=["accuracy"])
model_4.fit(X_train,Y_train,epochs=3,verbose=0)

Y_pred=[]
for i in range(X_test.shape[0]):
    num=model_4.predict(tf.expand_dims(X_test[i],0),verbose=0).argmax()
    Y_pred.append(num)

metrics=Eval_metrics(Y_test,Y_pred,10)
evaluation_summary('Model_4',metrics)

In [ ]:
model_5=Sequential([
    Flatten(input_shape=(28,28)),
    Dense(100,activation=relu),
    Dense(100,activation=relu),
    Dense(10,activation=sigmoid)
])

model_5.compile(optimizer='adam',loss=SparseCategoricalCrossentropy(),metrics=["accuracy"])
model_5.fit(X_train,Y_train,epochs=3,verbose=0)

Y_pred=[]
for i in range(X_test.shape[0]):
    num=model_5.predict(tf.expand_dims(X_test[i],0),verbose=0).argmax()
    Y_pred.append(num)

metrics=Eval_metrics(Y_test,Y_pred,10)
evaluation_summary('Model_5',metrics)

Model Name: Model_5

Accuracy: 0.799

Confusion_Matrix:
 [[ 82.   0.   1.   0.   0.   0.  10.   0.   0.   0.]
 [  0. 124.  28.   0.   0.   3.   0.   0.   0.   0.]
 [  0.   0.  81.  49.   2.   7.   0.   1.   1.   1.]
 [  2.   0.   2.  56.   0.  37.   0.   4.   3.   7.]
 [  0.   0.   0.   0. 101.   1.   1.   0.   2.   2.]
 [  0.   0.   1.   1.   0.  36.   3.   1.   1.   4.]
 [  1.   0.   0.   0.   2.   0.  72.   0.   0.   0.]
 [  0.   0.   1.   1.   0.   0.   0.  90.   1.   0.]
 [  0.   2.   1.   0.   1.   2.   1.   2.  80.   3.]
 [  0.   0.   1.   0.   4.   1.   0.   1.   1.  77.]]


In [ ]:
model_6=Sequential([
    Flatten(input_shape=(28,28)),
    Dense(100,activation=relu),
    Dense(10,activation=softmax)
])

model_6.compile(optimizer='adam',loss=SparseCategoricalCrossentropy(),metrics=["accuracy"])
model_6.fit(X_train,Y_train,epochs=3,verbose=0)

Y_pred=[]
for i in range(X_test.shape[0]):
    num=model_6.predict(tf.expand_dims(X_test[i],0),verbose=0).argmax()
    Y_pred.append(num)

metrics=Eval_metrics(Y_test,Y_pred,10)
evaluation_summary('Model_6',metrics)

Model Name: Model_6

Accuracy: 0.917

Confusion_Matrix:
 [[ 83.   0.   0.   1.   1.   1.   4.   1.   3.   0.]
 [  0. 124.   0.   0.   1.   0.   0.   0.   0.   0.]
 [  0.   0. 104.   1.   1.   0.   0.   0.   1.   0.]
 [  1.   0.   1. 101.   0.   2.   0.   3.   1.   0.]
 [  0.   0.   0.   0.  97.   1.   0.   1.   2.   5.]
 [  0.   1.   1.   2.   0.  75.   1.   0.   1.   0.]
 [  1.   0.   0.   0.   1.   3.  82.   0.   1.   0.]
 [  0.   0.   4.   0.   1.   1.   0.  92.   0.   1.]
 [  0.   1.   5.   1.   1.   3.   0.   0.  75.   4.]
 [  0.   0.   1.   1.   7.   1.   0.   2.   5.  84.]]


In [ ]:
model_7=Sequential([
    Flatten(input_shape=(28,28)),
    Dense(150,activation=relu),
    Dense(10,activation=softmax)
])

model_7.compile(optimizer='adam',loss=SparseCategoricalCrossentropy(),metrics=["accuracy"])
model_7.fit(X_train,Y_train,epochs=3,verbose=0)

Y_pred=[]
for i in range(X_test.shape[0]):
    num=model_7.predict(tf.expand_dims(X_test[i],0),verbose=0).argmax()
    Y_pred.append(num)

metrics=Eval_metrics(Y_test,Y_pred,10)
evaluation_summary('Model_7',metrics)

In [ ]:
model_8=Sequential([
    Flatten(input_shape=(28,28)),
    Dense(150,activation=tanh),
    Dense(10,activation=softmax)
])

model_8.compile(optimizer='adam',loss=SparseCategoricalCrossentropy(),metrics=["accuracy"])
model_8.fit(X_train,Y_train,epochs=3,verbose=0)

Y_pred=[]
for i in range(X_test.shape[0]):
    num=model_8.predict(tf.expand_dims(X_test[i],0),verbose=0).argmax()
    Y_pred.append(num)

metrics=Eval_metrics(Y_test,Y_pred,10)
evaluation_summary('Model_8',metrics)

In [ ]:
model_9=Sequential([
    Flatten(input_shape=(28,28)),
    Dense(150,activation=sigmoid),
    Dense(10,activation=softmax)
])

model_9.compile(optimizer='adam',loss=SparseCategoricalCrossentropy(),metrics=["accuracy"])
model_9.fit(X_train,Y_train,epochs=3,verbose=0)

Y_pred=[]
for i in range(X_test.shape[0]):
    num=model_9.predict(tf.expand_dims(X_test[i],0),verbose=0).argmax()
    Y_pred.append(num)

metrics=Eval_metrics(Y_test,Y_pred,10)
evaluation_summary('Model_9',metrics)

In [ ]:
model_10=Sequential([
    Flatten(input_shape=(28,28)),
    Dense(100,activation=sigmoid),
    Dense(10,activation=softmax)
])

model_10.compile(optimizer='adam',loss=SparseCategoricalCrossentropy(),metrics=["accuracy"])
model_10.fit(X_train,Y_train,epochs=3,verbose=0)

Y_pred=[]
for i in range(X_test.shape[0]):
    num=model_10.predict(tf.expand_dims(X_test[i],0),verbose=0).argmax()
    Y_pred.append(num)

metrics=Eval_metrics(Y_test,Y_pred,10)
evaluation_summary('Model_10',metrics)

In [ ]:
model_11=Sequential([
    Flatten(input_shape=(28,28)),
    Dense(100,activation=sigmoid),
    Dense(150,activation=tanh),
    Dense(10,activation=softmax)
])

model_11.compile(optimizer='adam',loss=SparseCategoricalCrossentropy(),metrics=["accuracy"])
model_11.fit(X_train,Y_train,epochs=3,verbose=0)

Y_pred=[]
for i in range(X_test.shape[0]):
    num=model_11.predict(tf.expand_dims(X_test[i],0),verbose=0).argmax()
    Y_pred.append(num)

metrics=Eval_metrics(Y_test,Y_pred,10)
evaluation_summary('Model_11',metrics)

In [ ]:
model_12=Sequential([
    Flatten(input_shape=(28,28)),
    Dense(100,activation=sigmoid),
    Dense(150,activation=tanh),
    Dense(10,activation=softmax)
])

model_12.compile(optimizer='sgd',loss=SparseCategoricalCrossentropy(),metrics=["accuracy"])
model_12.fit(X_train,Y_train,epochs=3,verbose=0)

Y_pred=[]
for i in range(X_test.shape[0]):
    num=model_12.predict(tf.expand_dims(X_test[i],0),verbose=0).argmax()
    Y_pred.append(num)

metrics=Eval_metrics(Y_test,Y_pred,10)
evaluation_summary('Model_12',metrics)

KeyboardInterrupt: 

In [ ]:
model_13=Sequential([
    Flatten(input_shape=(28,28)),
    Dense(150,activation=relu),
    Dense(150,activation=relu),
    Dense(10,activation=softmax)
])

model_13.compile(optimizer='adam',loss=SparseCategoricalCrossentropy(),metrics=["accuracy"])
model_13.fit(X_train,Y_train,epochs=3,verbose=0)

Y_pred=[]
for i in range(X_test.shape[0]):
    num=model_13.predict(tf.expand_dims(X_test[i],0),verbose=0).argmax()
    Y_pred.append(num)

metrics=Eval_metrics(Y_test,Y_pred,10)
evaluation_summary('Model_13',metrics)

In [ ]:
model_14=Sequential([
    Flatten(input_shape=(28,28)),
    Dense(150,activation=relu),
    Dense(150,activation=relu),
    Dense(10,activation=softmax)
])

model_14.compile(optimizer='adam',loss=SparseCategoricalCrossentropy(),metrics=["accuracy"])
model_14.fit(X_train,Y_train,epochs=3,verbose=0)

Y_pred=[]
for i in range(X_test.shape[0]):
    num=model_14.predict(tf.expand_dims(X_test[i],0),verbose=0).argmax()
    Y_pred.append(num)

metrics=Eval_metrics(Y_test,Y_pred,10)
evaluation_summary('Model_14',metrics)

In [ ]:
model_14=Sequential([
    Flatten(input_shape=(28,28)),
    Dense(150,activation=tanh),
    Dense(150,activation=tanh),
    Dense(10,activation=softmax)
])

model_14.compile(optimizer='adam',loss=SparseCategoricalCrossentropy(),metrics=["accuracy"])
model_14.fit(X_train,Y_train,epochs=3,verbose=0)

Y_pred=[]
for i in range(X_test.shape[0]):
    num=model_14.predict(tf.expand_dims(X_test[i],0),verbose=0).argmax()
    Y_pred.append(num)

metrics=Eval_metrics(Y_test,Y_pred,10)
evaluation_summary('Model_14',metrics)

In [ ]:
model_15=Sequential([
    Flatten(input_shape=(28,28)),
    Dense(150,activation=tanh),
    Dense(150,activation=tanh),
    Dense(10,activation=softmax)
])

model_15.compile(optimizer='adam',loss=SparseCategoricalCrossentropy(),metrics=["accuracy"])
model_15.fit(X_train,Y_train,epochs=3,verbose=0)

Y_pred=[]
for i in range(X_test.shape[0]):
    num=model_15.predict(tf.expand_dims(X_test[i],0),verbose=0).argmax()
    Y_pred.append(num)

metrics=Eval_metrics(Y_test,Y_pred,10)
evaluation_summary('Model_15',metrics)